Import required tools for collecting and cleaning the data: <br>
* Some tools are more commonly used
* Some tools are specific to the Alpha Vantage API

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
ALPHA_VANTAGE_API_KEY = os.getenv('secret')

In [2]:
import json
import requests
import collections
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
import numpy as np
import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import scipy.stats as sp
import pickle
import time

In [3]:
from alpha_vantage.foreignexchange import ForeignExchange
from alpha_vantage.timeseries import TimeSeries

import asyncio
from alpha_vantage.async_support.timeseries import TimeSeries

Obtain forex data from the Alpha Vantage API: <br>
* I am obtaining pricing data for the past 100 days for pairings of the top ten most traded currencies <br>
* For each pairing I am taking only the data required into a pandas dataframe, adding a column for "to" and "from" symbol to keep track of each currency, and then concatenating the dataframes into a large dataframe for easier use

* Assign API call to a variable "app"

In [4]:
app = ForeignExchange(key=ALPHA_VANTAGE_API_KEY, output_format='pandas')

* Create empty dataframe

In [5]:
all_pairs = pd.DataFrame()

Here we initialize two lists:
* CURRENCY_1: the currency we are converting FROM
* CURRENCY_2: the currency we are converting TO

In [6]:
CURRENCY_1 = ('USD', 'EUR', 'JPY', 'GBP', 'AUD', 'CHF', 'CAD', 'HKD', 'SEK', 'NZD',)
CURRENCY_2 = ('USD', 'EUR', 'JPY', 'GBP', 'AUD', 'CHF', 'CAD', 'HKD', 'SEK', 'NZD',)

Create the for loop:
* Double for loop to iterate through CURRENCY_1 and CURRENCY_2
* If statement to be sure that currency is not converted to itself
* Restrict data to usable part returned by API
* Create columns for "to symbol" and "from symbol"
* Rename columns for ease of use
* Convert to datetime, sort index
* Add features
* Reorder columns
* Concatenate to main dataframe

In [7]:
for c1 in CURRENCY_1:
  for c2 in CURRENCY_2:
    if c1 != c2:
        currconv = app.get_currency_exchange_daily(c1, c2)
        time.sleep(15)
        currconv1 = currconv[0]
        currconv1['to symbol'] = c2
        currconv1['from symbol'] = c1

        currconv1.rename(columns={'1. open': 'open'}, inplace=True)
        currconv1.rename(columns={'2. high': 'high'}, inplace=True)
        currconv1.rename(columns={'3. low': 'low'}, inplace=True)
        currconv1.rename(columns={'4. close': 'close'}, inplace=True)

        pd.to_datetime(currconv1.index)
        currconv1 = currconv1.sort_index()

        currconv1['time'] = np.arange(len(currconv1.index))

        currconv1['lag_1'] = currconv1['close'].shift(1)
        currconv1['lag_2'] = currconv1['close'].shift(2)
        currconv1['lag_3'] = currconv1['close'].shift(3)
        currconv1['lag_4'] = currconv1['close'].shift(4)
        currconv1['lag_5'] = currconv1['close'].shift(5)
        currconv1['lag_6'] = currconv1['close'].shift(6)
        currconv1['lag_7'] = currconv1['close'].shift(7)
        currconv1['lag_8'] = currconv1['close'].shift(8)
        currconv1['lag_9'] = currconv1['close'].shift(9)
        currconv1['lag_10'] = currconv1['close'].shift(10)
        currconv1['lag_11'] = currconv1['close'].shift(11)
        currconv1['lag_12'] = currconv1['close'].shift(12)
        currconv1['lag_13'] = currconv1['close'].shift(13)
        currconv1['lag_14'] = currconv1['close'].shift(14)

        currconv1['rolling_mean'] = currconv1['close'].rolling(window=7).mean()
        currconv1['expanding_mean'] = currconv1['close'].expanding(2).mean()

        currconv1 = currconv1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'lag_8', 'lag_9', 'lag_10', 'lag_11', 'lag_12', 'lag_13', 'lag_14', 'rolling_mean', 'expanding_mean', 'time']]

        all_pairs = pd.concat([all_pairs, currconv1])

* Add empty column

In [8]:
all_pairs['close_prediction'] = ''

* View dataframe

In [9]:
all_pairs

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,...,lag_9,lag_10,lag_11,lag_12,lag_13,lag_14,rolling_mean,expanding_mean,time,close_prediction
date,,,,,,,,,,,,,,,,,,,,,
2022-07-21,USD,EUR,0.9821,0.9838,0.9727,0.9773,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,
2022-07-22,USD,EUR,0.9775,0.9865,0.9750,0.9785,0.9773,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.977900,1,
2022-07-25,USD,EUR,0.9785,0.9821,0.9750,0.9781,0.9785,0.9773,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.977967,2,
2022-07-26,USD,EUR,0.9782,0.9891,0.9754,0.9880,0.9781,0.9785,0.9773,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.980475,3,
2022-07-27,USD,EUR,0.9880,0.9899,0.9783,0.9801,0.9880,0.9781,0.9785,0.9773,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.980400,4,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-01,NZD,SEK,6.5985,6.6487,6.5448,6.5644,6.6038,6.5576,6.5020,6.5051,...,6.5400,6.4981,6.4420,6.4535,6.3944,6.3358,6.538943,6.425739,95,
2022-12-02,NZD,SEK,6.5595,6.6357,6.5319,6.6017,6.5644,6.6038,6.5576,6.5020,...,6.5310,6.5400,6.4981,6.4420,6.4535,6.3944,6.549671,6.427553,96,
2022-12-05,NZD,SEK,6.5576,6.6273,6.5488,6.5694,6.6017,6.5644,6.6038,6.5576,...,6.5468,6.5310,6.5400,6.4981,6.4420,6.4535,6.557714,6.429000,97,
